## Part 2

## Main Import Commands

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd 
import transformers
from torch.utils.data import Dataset, DataLoader 
from tqdm.notebook import tqdm


from transformers import AutoTokenizer, AutoModel
sentenc_model_name = "sentence-transformers/paraphrase-MiniLM-L3-v2" # Can be changed to a different one found in https://www.sbert.net/docs/pretrained_models.html
tokenizer = AutoTokenizer.from_pretrained(sentenc_model_name) 

sum_dir = ""


## Problem Formulation and Model 

The problem is treated as a sentence classification problem where the goal is to classify each sentence in a document as belonging to a summary or not, given the document.

- Model Input: a sentence, and the entire document 
- Model Output: A score representing likelihood of belonging in the summary or not. 

## Implementation 

- The Sentence BERT model (with the pre-trained pipeline) is used to get text representations.
- Representations for sentence and document are concatenated and fed to a Dense layer and then predictions.
- See 'utils\sbert' for full script of the adapted Sentence BERT class.

In [ ]:
# Defining some key trainng variables
MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05 

# load dataframes containining preprocessed samples from the original portion of the dataset
train_df = pd.read_json(sum_dir + "data/train_bdf_da1860.json")
test_df = pd.read_json(sum_dir +"data/test_bdf_da1855.json") # The test-set remains constant across all experiments
print( "Train, test shape", train_df.shape, test_df.shape)
print( "Train, test shape", train_df.shape, test_df.shape)

In [ ]:
from utils.data_loader import* # Importing the 'Data_loader' class to load the data, yielding sentences, documents and labels


training_set = Data_loader(train_df, tokenizer, MAX_LEN)
testing_set = Data_loader(test_df, tokenizer, MAX_LEN)

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)        

In [ ]:
from utils.sbert import* # Importing the 'mean_pooling' function and the 'SentenceBertClass' class, serving a crucial part of the model
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

model = SentenceBertClass(model_name=sentenc_model_name)
model.to(device);

loss_function = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Defining the training function
print_n_steps = 1000
EPOCHS = 10
acc_step_holder, loss_step_holder = [], []


def train(epoch):    
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        sent_ids = data['sent_ids'].to(device, dtype = torch.long)
        doc_ids = data['doc_ids'].to(device, dtype = torch.long)
        sent_mask = data['sent_mask'].to(device, dtype = torch.long)
        doc_mask = data['doc_mask'].to(device, dtype = torch.long) 
        targets = data['targets'].to(device, dtype = torch.float)  

        outputs = model(sent_ids, doc_ids, sent_mask, doc_mask) 
        loss = loss_function(outputs, targets)
        tr_loss += loss.item() 
        n_correct += torch.count_nonzero(targets == (outputs > 0.5)).item()

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%print_n_steps==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(str(_* train_params["batch_size"]) + "/" + str(len(train_df)) + " - Steps. Acc ->", accu_step, "Loss ->", loss_step)
            acc_step_holder.append(accu_step), loss_step_holder.append(loss_step)
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        break

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

In [ ]:
# Plotting training accuracy and loss (would be more useful for a more comprehensive training process)

fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,5))
ax1.plot(acc_step_holder, label="Accuracy")
ax2.plot(loss_step_holder, label="Loss")
ax1.title.set_text("Accuracy")
ax2.title.set_text("Loss")
fig.tight_layout()
plt.show()

## Save Model

In [ ]:
os.makedirs("models", exist_ok=True)
torch.save(model.state_dict(), "models/L03_1860.pth")
!gsutil cp -r models $sum_dir